In [6]:
import os, sys
import sqlite3
import pandas as pd
cwd = os.getcwd()
root_path = os.path.join(cwd, '../')
db_name = 'rtcw_db.db'

In [7]:
db = sqlite3.connect(os.path.join(root_path, db_name))

In [8]:
cursor = db.cursor()

In [5]:
cursor.execute('select * from demo')
cursor.fetchall()

[(1,
  u'6f7605523ed948dcb2ea7090afbcc447',
  u'1055818784d@n!el-mp_ice',
  u'',
  u'[QCON]Nate',
  u'',
  u'',
  u'',
  2296,
  16,
  466,
  0,
  0,
  0,
  0,
  0),
 (2,
  u'8726f959cb5c981ba964da7a5d7741b3',
  u'SM-Ligan_Merge-SWF_POV_s3VVi_Beach',
  u'',
  u'',
  u'',
  u'',
  u'',
  1860,
  12,
  313,
  0,
  0,
  0,
  0,
  0),
 (3,
  u'59b1f2608f22e9e90e944b36f5929c7c',
  u'cohvstfp',
  u'',
  u'CoH]I[Smoove',
  u'',
  u'',
  u'',
  3329,
  13,
  0,
  0,
  0,
  0,
  0,
  0),
 (4,
  u'1aac0340671f5d2c5e4ff47dbd9e4276',
  u'043942-grimdeathHV-mp_ice',
  u'',
  u'WTV-Speakeasy.NET',
  u'',
  u'',
  u'',
  468,
  13,
  134,
  0,
  0,
  0,
  0,
  0),
 (5,
  u'd8b6a9deef488cf7e8b560723047e80d',
  u'045124-silenceHV-mp_ice',
  u'',
  u'WTV-Speakeasy.NET',
  u'',
  u'',
  u'',
  750,
  13,
  232,
  0,
  0,
  0,
  0,
  0),
 (6,
  u'812a11734eef9fe1a7e03630aba0d4ce',
  u'050920-silentstormHV-mp_ice',
  u'',
  u'WTV-Speakeasy.NET',
  u'',
  u'',
  u'',
  500,
  13,
  151,
  0,
  0,
  0,
  0,
